# Non-communicatingなMDPにおける探索と活用について

参考：
* [Near Optimal Exploration-Exploitation in Non-Communicating Markov Decision Processes](https://arxiv.org/abs/1807.02373)

---

前知識（Weakly-communicating MDP）
* MDP: $M=\langle\mathcal{S}, \mathcal{A}, r, p\rangle$
  * $\mathcal{A}=\bigcup_{s \in \mathcal{S}} \mathcal{A}_s$

Weakly-communicating MDPでは，状態空間が２種類に分割されます．
1. Communicatingな状態空間 $\mathcal{S}^C$：$\mathcal{S}^C$の任意の状態が，他の任意の状態から遷移可能であるような方策が存在する．
2. Transientな状態空間 $\mathcal{S}^T$：$s \in \mathcal{S}^T$な$s$について，任意の決定的な定常方策に対して，$s$から再び$s$に戻ってくる確率が$<1$である．（これはつまり，transient stateは有限回しか訪問されないことを意味します（多分一度Transientから出ると，もう戻ってこれない＆CommunicatingからTransientには渡れない？）．また，MDPの半径が$D=+\infty$になります）

補足：今回の話はMulti-chain MDPってやつに簡単にスケールします．TODOにしておきます．

---

また，今回は初期状態が$\mathcal{S}^C$にあるとします．このとき，$\mathcal{S}^T$の状態空間は絶対に訪問しません．
1. $s_T \in \mathcal{S}^T$を訪問する方策$\pi$があるとしましょう．
2. $\mathcal{S}^T$はTransientなので，どの方策でも$s_T$から$\mathcal{S}^C$に帰ってきます．
3. $\mathcal{S}^C$はcommunicatingなので，再び初期状態に戻れます．
4. そして$\pi$を使うと$s_T$に到達できてしまいます．よって，無限回訪問することが可能であり，これはtransientの定義と矛盾します．

---

今回は無限ホライゾン平均報酬を考えます．
無限ホライゾン平均報酬を解く有名なアルゴリズムとしてUCRLアルゴリズムがあります（https://arxiv.org/abs/1205.2661 など参照）．
UCRLアルゴリズムなどは，MDPの半径が$D < \infty$であることを仮定します．
これはつまり，任意の状態から他の状態に有限ステップ以内に到達できることを仮定していますが，このような半径の仮定は実践では成り立たないことが多いです
（論文中のMountain Carの図参照）．

Weakly communicating MDPなどでは，UCRLアルゴリズムは到達不可能な状態をずっと探そうとしてしまいます．
半径の代わりにspanが$H$でバウンドされていることを利用するアルゴリズム（REGAL.Cなど）も存在しますが，これも到達不可能な状態を排除していることに他なりません．
また，spanのバウンドの事前知識や半径の事前知識を持つことは現実的ではありません．
REGAL.Dはspanの事前知識が不要ですが，理論が間違ってるぽいです．
ちなみに有限ホライゾンの場合でもweakly communicating MDPで生じる「到達不可能な状態をずっと探そうとする」問題は起こりえません．

今回は，到達不可能な状態空間についての事前知識がない場合のアルゴリズムについて見てみます．


---

準備：
* 平均報酬：$g_M^\pi(s):=\lim _{T \rightarrow \infty} \mathbb{E}\left[\frac{1}{T} \sum_{t=1}^T r\left(s_t, \pi\left(s_t\right)\right)\right]$
* バイアス関数：$h_M^\pi(s):=\underset{T \rightarrow \infty}{C\text{-}\lim _T} \mathbb{E}\left[\sum_{t=1}^T\left(r\left(s_t, \pi\left(s_t\right)\right)-g_M^\pi\left(s_t\right)\right)\right]$
* Weakly communicating MDPでは，任意の最適方策$\pi^* \in \arg \max _\pi g^\pi(s)$は定数ゲインを持ちます．つまり，$g^{\pi^*}(s)=g^*$が任意の$s$で成立します．
* $D$：MDPの半径.$D:=\max _{\left(s, s^{\prime}\right) \in \mathcal{S} \times \mathcal{S}, s \neq s^{\prime}}\left\{\tau_M\left(s \rightarrow s^{\prime}\right)\right\}$
* $D^C$：Communicatingな部分の半径．$D^{\mathrm{C}}:=\max _{\left(s, s^{\prime}\right) \in \mathcal{S}^{\mathrm{C}} \times \mathcal{S}^{\mathrm{C}}, s \neq s^{\prime}}\left\{\tau_M\left(s \rightarrow s^{\prime}\right)\right\}$
* $\tau_M\left(s \rightarrow s^{\prime}\right)$：MDP $M$において$s$から$s'$に到達するまでにかかる期待ステップ数
* リグレット：$\Delta(\mathfrak{A}, T)=\sum_{t=1}^T g^*-r_t\left(s_t, a_t\right)$.
    * ここで$\mathcal{U}$は何らかのアルゴリズムです．
* 仮定：$s_1$はcommunicating な$\mathcal{S}^C$に入っているとします．
* 仮定：状態空間と行動空間は既知とします．論文だと報酬は未知ですが，面倒なので既知とします．

* $t_k$：エピソード開始時のステップ
* エピソード$k$：エピソードは最低１ステップ進んだ後，次のどちらかが発生した場合に終了します．
    * 今まで見たことない状態に訪れる．これをすることで，$\mathcal{S}^T_k$に入ってる状態行動については，必ず$N_{k+1}(s, a)=0$になります．
    * エピソード開始時までに訪れた回数$N(s, a)$を再びそのエピソードで訪問する．（倍々のエピソード数にすると，全体で生じるエピソードの数がバウンドできます）
* $N_k(s, a)$は$k$エピソード以前に$(s, a)$を訪問した回数です．
* $N_k^{+}(s, a):=\max \left\{1, N_k(s, a)\right\}$と$N_k^{ \pm}(s, a):=\max \left\{1, N_k(s, a)-1\right\}$です．

* 信頼区間：
$$
\begin{aligned}
B_{k}(s, a) & =\left\{\widetilde{p}(\cdot \mid s, a) \in \Delta{S}: \forall s^{\prime} \in \mathcal{S},\left|\widetilde{p}\left(s^{\prime} \mid s, a\right)-\widehat{p}\left(s^{\prime} \mid s, a\right)\right| \leq \beta_{k}^{s a s^{\prime}}\right\} 
\end{aligned}
$$
ここで，
$$
\beta_{k}^{s a s^{\prime}}:=\sqrt{\frac{14 \widehat{\sigma}_{k}^2\left(s^{\prime} \mid s, a\right) b_{k, \delta}}{N_k^{+}(s, a)}}+\frac{\frac{49}{3} b_{k, \delta}}{N_k^{ \pm}(s, a)}
$$
* $\mathcal{M}_k=\left\{M=(\mathcal{S}, \mathcal{A}, r, \widetilde{p}): \widetilde{p}(\cdot \mid s, a) \in B_{p, k}(s, a)\right\}$とします．


## TUCRLアルゴリズム

TUCRLアルゴリズムはWeakly communicating MDPにおいて，事前情報を必要としないアルゴリズムです．
元論文では報酬の信頼区間も作ってますが，今回は無視します．

さて，UCRLアルゴリズムはMDPの集合$\mathcal{M}_k$について楽観的に方策を選択するわけですが，この場合，到達不可能な状態空間をずっと探索しかねません．
これを回避するために，TUCRLでは最初に$\mathcal{S}^C$と$\mathcal{S}^T$を推定して構築します．具体的には，

$$
\mathcal{S}_k^{\mathrm{C}}:=\left\{s \in \mathcal{S} \mid \sum_{a \in \mathcal{A}_s} N_k(s, a)>0\right\} \cup \{s_{t_k}\}
$$
および
$$
\mathcal{S}_k^{\mathrm{T}}:=\mathcal{S} \backslash \mathcal{S}_k^{\mathrm{C}}
$$
とします．
つまり，$\mathcal{S}^C$は$k$エピソード目までに1度でも訪問された状態です
（Transientは永遠に再訪問しない確率が少しあるので，もしTransientに訪問してしまうと，それはCommunicatingな状態から訪問可能なので，必ず再訪問できることになり，Transientと矛盾します）．

これを使って，TUCRLでは$\mathcal{S}^C_k$だけを探索することを考えます．
$s_1 \in \mathcal{S}^C$なので，$\mathcal{S}_k^C \subseteq \mathcal{S}^C$が必ず成立します．

---

しかし，このままでは「$\mathcal{S}^C$にあるが，$\mathcal{S}^C_k$にはない状態」を探索することができません．そこで，
* $\mathcal{S}^C_k$から$\mathcal{S}^T_k$に遷移する確率の下界を考え，
* もし「$s$から$s'$へ行く信頼区間の上界」がその下界以下であれば，その遷移は無視します．
* ここで，絶対に$\mathcal{S}^T$に訪問する確率が０なので，経験確率 $\hat{p}_k(s'\mid s, a)$と $\hat{\sigma}^2_{p,k}(s'\mid s, a)$は必ず０になります． よって， OptimisticなMDPにおける遷移は最大でも $\tilde{p}_k^{+}\left(s^{\prime} \mid s, a\right)=\frac{49}{3} \frac{b_{k, \delta}}{N_k^{ \pm}(s, a)}$ になります．この値が，後で決める$\rho_{t_k}$以下であれば，その遷移は無視することにします．

これを踏まえて，次のような信頼区間を考えましょう．つまり，上で説明した訪れそうにない遷移は除外して，その確率を０にします．

$$
\bar{B}_{p, k}(s, a)=B_{p, k}(s, a) \cap\left\{\widetilde{p}(\cdot \mid s, a) \in \mathcal{C}: \forall s^{\prime} \in \mathcal{S}_k^{\mathrm{T}} \text { and } \widetilde{p}_k^{+}\left(s^{\prime} \mid s, a\right)<\rho_{t_k}, \widetilde{p}\left(s^{\prime} \mid s, a\right)=0\right\}
$$

* また，$\bar{\mathcal{M}}_{k}$を，$\bar{B}_{p, k}$で作られるMDPの集合とします．
* （技術的な理由から，論文中では
$B_{p, k}^{+}(s, a)=\left\{\widetilde{p}(\cdot \mid s, a) \in \mathcal{C}:\|\widetilde{p}(\cdot \mid s, a)-\widehat{p}(\cdot \mid s, a)\|_1 \leq \sum_{s^{\prime}} \beta_{p, k}^{s a s^{\prime}}\right\}$なる信頼区間と，それに対応するMDP $\mathcal{M}^+_k$を考えてます．）今回は面倒なので$\mathcal{M}_k$のままで良いでしょう）．


TUCRLはこの信頼区間に対してOptimisticな方策を選択します．

$$
\left(\widetilde{M}_k, \widetilde{\pi}_k\right)=\underset{M \in \overline{\mathcal{M}}_k, \pi}{\arg \max }\left\{g_M^\pi\right\}
$$

また，
$$
\rho_t=\frac{49 b_{t, \delta}}{3} \sqrt{\frac{S A}{t}}
$$
とします．つまり，ある$(s, a, s')$において，$s'$に訪問したことがないが，さらに
$$N_k^{ \pm}(s, a)>\sqrt{t_k / S A}$$
である場合は$s'$を無視します．

* 以降，
$$
\mathcal{K}_k=\left\{(s, a) \in \mathcal{S}_k^{\mathrm{C}} \times \mathcal{A}: N_k^{ \pm}(s, a) \leq \sqrt{t_k / S A}\right\}
$$
を，$\mathcal{S}^C_k$のうち，十分に探索されていない状態行動の集合とします（つまり，これはまだ$s'$が$0$だとしても，$\mathcal{S}^T$だと判断するには早すぎる）．



まとめると，TUCRLでは次の処理を行います：

各エピソード$k=1, 2, \dots$で，
1. $t_k=t$，$\nu_k(s, a)=0$
2. $\widehat{p}_k\left(s^{\prime} \mid s, a\right)$および集合$\overline{\mathcal{M}}_k$を計算します
3. 楽観的なMDPと方策を計算します：$\left(\widetilde{M}_k, \widetilde{\pi}_k\right)=\underset{M \in \overline{\mathcal{M}}_k, \pi}{\arg \max }\left\{g_M^\pi\right\}$
4. 次が起きたらWhile loopを抜ける → 1. 今まで見たことない状態に訪れる．2. エピソード開始時までに訪れた回数$N(s, a)$を再びそのエピソードで訪問する．
    1. $a_t = \tilde{\pi}_k(s_t)$を実行し，報酬と次状態を得ます
    2. $\nu_k\left(s_t, a_t\right)+=1$ and set $t+=1$
5. $N_{k+1}(s, a)=N_k(s, a)+\nu_k(s, a)$


## リグレット解析

TUCRLは高確率で次のリグレットを達成します：

$$
\Delta(\mathrm{TUCRL}, T)=O\left(r_{\max } D^c \sqrt{\Gamma^c S^c A T \ln \left(\frac{S A T}{\delta}\right)}+r_{\max }\left(D^c\right)^2 S^3 A \ln ^2\left(\frac{S A T}{\delta}\right)\right)
$$

これを示しましょう．

### ステップ１：リグレットの分解

リグレットを思い出しましょう．
$$\Delta(\mathrm{TUCRL}, T):=\sum_{t=1}^T g^* - r_t\left(s_t, a_t\right)$$

ここで，
$$\Delta_k=\sum_{s \in \mathcal{S}, a \in \mathcal{A}} \nu_k(s, a)\left(g^*-r(s, a)\right)$$
とすると（ここで$\nu_k(s, a)$はエピソード$k$中で訪問した$(s, a)$の回数です），
$$
\Delta(\mathrm{TUCRL}, T) \leq \sum_{k=1}^m \Delta_k+r_{\max } \sqrt{\frac{5}{2} T \ln \left(\frac{8 T}{\delta}\right)}
$$
が確率$1-\frac{\delta}{12 T^{4 / 5}}$以上で成立します（これは[RL_AverageReward_UCRL2.ipynb](RL_AverageReward_UCRL2.ipynb)と同じやり方で出てきます）．

### $M^* \in \mathcal{M}_k$なエピソード

$\overline{\mathcal{M}}_k$ではTransientな状態を排除するので，$M^* \notin \overline{\mathcal{M}}_k$の場合があります．
例えばそもそもアルゴリズム開始時は$\mathcal{S}^C=\emptyset$なので，「全ての状態が到達不可能」だと考えられており，明らかに$M^*$が入りません．

これでは$S^T_k\neq \emptyset$のとき，OptimismをUCRL2みたいに証明することができません．そこで，エピソードの開始時刻$t_k$が次のように十分大きいとします：

$$
t_k \geq \frac{2401}{9}\left(D^{\mathrm{c}}\right)^2 S A\left(S_k^{\mathrm{T}} \ln \left(\frac{2 S A t_k}{\delta}\right)\right)^2:=C(k)
$$

このとき，次が成立します：

---

$k$を
* $M^* \in \mathcal{M}_k$：真のMDPが**排除する前の**信頼集合に入る
* $\mathcal{S}^T_k\neq \emptyset$：Transientな状態が存在する．
* $t_k \geq C(k)$：$t_k$が十分大きい．

であるエピソードとします．

このとき，
$$
\left(\forall(s, a) \in \mathcal{S}_k^c \times \mathcal{A}, N_k^{ \pm}(s, a)>\sqrt{\frac{t_k}{S A}}\right) \Longrightarrow \mathcal{S}_k^T=\mathcal{S}^T
$$
が成り立ちます．すなわち，次のどちらかが成り立っています：

1. $S_k^T=S^T$．
2. $\mathcal{K}_k\neq \emptyset$．つまり，$(s, a)\in \mathcal{S}^C_k\times \mathcal{S}$から$\mathcal{S}_k^T$へ行く状態行動が存在します．つまり，本来行けない状態を考慮しています．

これは$\mathcal{S}_k^T \supset \mathcal{S}^T$の状況を排除，つまり「本来は行けるのに行けないと考えてしまう状態」がある状況を排除し，Optimismを成立させます．

丁寧な議論は17ページあたりに任せますが，とにかくOptimismが成立します．

**証明**

$t_k$が十分大きいとして，さらに
$$
N_k^{ \pm}(s, a)>\sqrt{\frac{t_k}{S A}} \geq \frac{49}{3} D^{\mathrm{C}} S_k^{\mathrm{T}} \ln \left(\frac{2 S A t_k}{\delta}\right)
$$
が成り立っているとしましょう．仮定より，任意の$\left(s, a, s^{\prime}\right) \in \mathcal{S}_k^{\mathrm{C}} \times \mathcal{A} \times \mathcal{S}_k^{\mathrm{T}}$について，

$$
\begin{aligned}
\underbrace{p\left(s^{\prime} \mid s, a\right)}_{\text {transition probability in } M^*} & \leq \underbrace{\widehat{p}_k\left(s^{\prime} \mid s, a\right)}_{=0}+\beta_k^{s a s^{\prime}}=\underbrace{\sqrt{\frac{14 \widehat{\sigma}_{k}^2\left(s^{\prime} \mid s, a\right) \ln \left(2 S A t_k / \delta\right)}{N_k^{+}(s, a)}}+\frac{49 \ln \left(2 S A t_k / \delta\right)}{3 N_k^{ \pm}(s, a)}}_{=0} \\
& \leq \frac{49 \ln \left(2 S A t_k / \delta\right)}{3 N_k^{ \pm}(s, a)}<\frac{1}{D^{\mathrm{c}} S_k^{\mathrm{T}}}
\end{aligned}
$$

が成り立ちます．ここで，$\widehat{p}\left(s^{\prime} \mid s, a\right)=0$ and $\widehat{\sigma}_{k}^2\left(s^{\prime} \mid s, a\right)=0$をつかいました．

適当な$\bar{s} \in \mathcal{S}^T_k$を考えます．そして，
$\tau(s):=\tau_{M^*}(s \rightarrow \bar{s})$
および
$\tau_{\min }:=\min _{s \in \mathcal{S}_k^c}\{\tau(s)\}$
とします．このとき，

$$
\begin{aligned}
\tau(\bar{s}) & =0 \\
\forall s \in \mathcal{S}_k^{\mathrm{C}} \quad \tau(s) & =1+\min _{a \in \mathcal{A}_s}\{\sum_{s^{\prime} \in \mathcal{S}} \underbrace{p\left(s^{\prime} \mid s, a\right) \tau\left(s^{\prime}\right)}_{\geq 0}\} \geq 1+\min _{a \in \mathcal{A}_s}\{\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{c}}} p\left(s^{\prime} \mid s, a\right) \underbrace{\tau\left(s^{\prime}\right)}_{\geq \tau_{\min }}\} \\
& \geq 1+\tau_{\min } \cdot \min _{a \in \mathcal{A}}\left\{\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{C}}} p\left(s^{\prime} \mid s, a\right)\right\}=1+\tau_{\min } \cdot \min _{a \in \mathcal{A}}\{1-\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{T}}} \underbrace{p\left(s^{\prime} \mid s, a\right)}_{<\frac{1}{D^{\mathrm{c}} S_k^{\mathrm{T}}}}\} \\
& >1+\tau_{\min }\left(1-\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{T}}} \frac{1}{D^{\mathrm{C}} S_k^{\mathrm{T}}}\right)=1+\tau_{\min }\left(1-\frac{1}{D^{\mathrm{C}}}\right)
\end{aligned}
$$

が成り立ちます．また，これを使うと$\tau_{\min }>D^{\mathrm{C}}$もわかります．
よって，
1. $M^*$における任意の$s \in \mathcal{S}_k^{\mathrm{C}} \subseteq \mathcal{S}^{\mathrm{C}}$と$\bar{s} \in \mathcal{S}_k^{\mathrm{T}}$間の最短距離は$D^c$より大きいことになります
2. しかし，$D^C$はCommunicating内の最長の最短距離なので，$\bar{s}$はTransientでないといけません．

よって，$\bar{s} \in S^T$です．

---



上の議論をベースに，$t_k \gg 1$を考えて，Optimismがとりあえず成立してるとしましょう．

このとき，エピソードのリグレットは

$$
\begin{aligned}
& \Delta_k=\sum_{s, a} \nu_k(s, a)\left(g^*-r(s, a)\right)
\leq \underbrace{\sum_{s, a} \nu_k(s, a)\left(\widetilde{g}_k-r_k(s, a)\right)}_{:=\widetilde{\Delta}_k}
\end{aligned}
$$

で得られます．最後の項をバウンドしましょう．
まず，$\mathcal{K}_k$，つまりあまり精度がない状態行動を排除します：


$$
\begin{aligned}
\widetilde{\Delta}_k & =\sum_{s, a} \nu_k(s, a)\left(\widetilde{g}_k-r(s, a)\right) \underbrace{\mathbb{1}\left\{(s, a) \notin \mathcal{K}_k\right\}}_{:=\mathbb{1}_k(s, a)}+\sum_{s, a} \nu_k(s, a) \underbrace{\left(\widetilde{g}_k-r(s, a)\right)}_{\leq r_{\max }} \mathbb{1}\left\{(s, a) \in \mathcal{K}_k\right\} \\
& \leq \underbrace{\sum_{s, a} \nu_k(s, a)\left(\widetilde{g}_k-r(s, a)\right) \mathbb{1}_k(s, a)}_{:=\widetilde{\Delta}_k^{\prime}}+r_{\max } \sum_{s, a} \nu_k(s, a) \mathbb{1}\left\{(s, a) \in \mathcal{K}_k\right\}
\end{aligned}
$$

ベルマン方程式を使えば，
$$
\begin{aligned}
\widetilde{\Delta}_k^{\prime}
&=\sum_s \nu_k\left(s, \widetilde{\pi}_k(s)\right)\left(\widetilde{g}_k-r\left(s, \widetilde{\pi}_k(s)\right)\right) \mathbb{1}_k\left(s, \widetilde{\pi}_k(s)\right)\\
&=\sum_s \nu_k\left(s, \widetilde{\pi}_k(s)\right)
\left(\sum_{s^{\prime} \in \mathcal{S}} \widetilde{p}_k\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) \widetilde{h}_k\left(s^{\prime}\right)-\widetilde{h}_k(s)\right) \mathbb{1}_k\left(s, \widetilde{\pi}_k(s)\right)\\
&=\nu_k^{\prime}\left(\widetilde{P}_k-I\right) \widetilde{h}_k
\end{aligned}
$$

です．ここで，$\widetilde{P}_k$は確率行列なので，
$
\left(\widetilde{P}_k-I\right) \widetilde{h}_k=\left(\widetilde{P}_k-I\right)\left(\widetilde{h}_k+\lambda e\right)
$
が任意の$\lambda$で成り立ちます．
ここで，

* $\lambda:=-\min _{s \in \mathcal{S}_k^{\mathrm{c}}}\left\{\widetilde{h}_k(s)\right\}$
* $w_k:=\widetilde{h}_k+\lambda e$．よって，これは０以上かつ$\min _{s \in \mathcal{S}_k^c}\left\{w_k(s)\right\}=0$です．

としましょう．
このとき，

$$
\nu_k^{\prime}\left(\widetilde{P}_k-I\right) \widetilde{h}_k=\sum_{s \in \mathcal{S}} \nu_k\left(s, \widetilde{\pi}_k(s)\right) \mathbb{1}_k\left(s, \widetilde{\pi}_k(s)\right)\left(\sum_{s^{\prime} \in \mathcal{S}} \widetilde{p}_k\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right)-w_k(s)\right)
$$
が成り立ちます．特に$\nu_k$は新しい状態に訪問するとリセットされるので，
$$
\nu_k^{\prime}\left(\widetilde{P}_k-I\right) \widetilde{h}_k=\sum_{s \in \mathcal{S}_k^c} \nu_k\left(s, \widetilde{\pi}_k(s)\right) \mathbb{1}_k\left(s, \widetilde{\pi}_k(s)\right)\left(\sum_{s^{\prime} \in \mathcal{S}} \widetilde{p}_k\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right)-w_k(s)\right)
$$
が成り立ちます．さらに，Transientな状態の排除の仕方から，

$$
\nu_k^{\prime}\left(\widetilde{P}_k-I\right) \widetilde{h}_k=\sum_{s \in \mathcal{S}_k^{\mathrm{C}}} \nu_k\left(s, \widetilde{\pi}_k(s)\right) \mathbb{1}_k\left(s, \widetilde{\pi}_k(s)\right)\left(\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{C}}} \widetilde{p}_k\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right)-w_k(s)\right)
$$

も成り立ってます．最後の（）の中身を分解しましょう．適当な$p$について，
$$
\begin{aligned}
\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{c}}} \widetilde{p}_k\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right)-w_k(s)= & \sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{c}}} \widetilde{p}_k\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right)-p\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right) \\
& +\left(\sum_{s^{\prime} \in \mathcal{S}_k^{\mathrm{c}}} p\left(s^{\prime} \mid s, \widetilde{\pi}_k(s)\right) w_k\left(s^{\prime}\right)-w_k(s)\right)
\end{aligned}
$$

が成り立ちます．ここで，

$$
\left\|\widetilde{p}_k-p\right\|_{1, \mathcal{S}^c} \leq\left\|\widetilde{p}_k-\widehat{p}_k\right\|_{1, \mathcal{S}^c}+\left\|\widehat{p}_k-p\right\|_{1, \mathcal{S}^c} \leq 2\left(\sum_{s^{\prime} \in \mathcal{S}^c} \beta_{k}^{s \tilde{s}_k(s) s^{\prime}}\right)
$$
を使えば，一項目はボーナスでバウンドできます．


**TODO: 続き書く**

## 補足：直径が無限の場合の探索について

ちょっとUCRLのリグレットと今回の話を比較してみましょう．
UCRLについては以下のリグレットが成り立ちます：
$$
\mathbb{E}[\Delta(\text { UCRL }, M, T)] \leq\left\{\begin{array}{l}
r_{\max } T \text { (by definition) } \\
C_1 \cdot r_{\max } D \sqrt{\Gamma S A T \ln \left(3 T^2\right)}+\frac{1}{3}[2, \text { Theorem 2] } \\
C_2 \cdot r_{\max } \frac{D^2 \Gamma S A}{\gamma} \ln (T)+C_3(M)[2, \text { Theorem 4 }]
\end{array}\right.
$$

ここで，$\delta = 1/3T$，
$\gamma=g_M^*-\max _{s, \pi}\left\{g_M^\pi(s): g_M^\pi(s)<g_M^*\right\}$はゲインのギャップ，$C_1$と$C_2$は定数です．
また，$C_3(M):=O\left(\max _{\pi: \pi(s)=a} T_\pi\right)$は方策$\pi$のMixing timeを測ってます．

この３つのリグレットをプロットすると，

![UCRL-regrets](figs/UCRL-regrets.png)

みたいになります．つまり，

1. $T_M^{\dagger} \geq T \geq 0$ では線形リグレット
2. $T_M^* \geq T \geq T_M^{\dagger}$では√Tリグレット
3. それ以降は対数リグレット

です．このような挙動は実験的にも観測されるみたいです．（[Efficient Bias-Span-Constrained Exploration-Exploitation in Reinforcement Learning](https://arxiv.org/abs/1802.04020)）

さて，上のリグレットより，劣線形リグレットに至るのは最悪でも$T_M^{\dagger}=\widetilde{O}\left(D^2 \Gamma S A\right)$ステップ後です．
このように，「劣線形リグレットに達するのがMDPのパラメーターに対して多項式時間後」の場合，「そのアルゴリズムはEfficientである」，と呼ぶことにします．

以下，「どの効率的なアルゴリズムも，前知識がなければ，$T^*_M=+\infty$になることを示します．つまり，対数リグレットは前知識なしでは無理です．


---

![no-logarithmic](figs/weakly-no-logarithmic.png)

上のMDPのクラス$\mathcal{M}$を考えてみましょう．違いは$x$から$y$に行く確率$\varepsilon$だけです．
* (b)は$\varepsilon > 0$です．最適方策は$\pi^*(x)=b$であり，最適ゲインは$g_\varepsilon^*=1$です．
    * $\varepsilon > 0$については $D=s p_{\mathcal{S}}\left\{h^*\right\}=1 / \varepsilon<+\infty$です．
* (c)は$\varepsilon =0$です．最適方策は$\pi^*(x)=d$であり，最適ゲインは$g_0^*=1/2$です．
    * $\varepsilon = 0$については $D=+\infty$ですが，$s p_{\mathcal{S}}\left\{h^*\right\}=1 / 2$です．．
* すべてのMDPに対して$S=2 \geq \Gamma$，$A=2$，そして平均報酬のギャップは$\gamma=1/2$です．よって，任意の$S, \Gamma, A, \gamma$についての関数は定数としてみなせます．


ここで，真のMDP $M^*$ が$\mathcal{M}$ に属することはわかってますが，その値$\varepsilon^*$ はわからないとします．
このとき，次が成立します：

$C_1, C_2, \alpha, \beta>0$は正の値とします．
$\varepsilon \in] 0,1]$に対して，
$f(\varepsilon)=C_1(1 / \varepsilon)^\alpha$を定義します．

このとき，既知のホライゾン$T$に対して，次を満たすアルゴリズム$\mathfrak{A}_T$は存在しません：
1. 任意の$\varepsilon \in] 0,1]$に対して，$\mathbb{E}\left[\Delta\left(\mathfrak{A}_T, M_{\varepsilon}, x, T\right)\right]<1 / 6 \cdot T$ for all $T \geq T_{\varepsilon}^{\dagger}$を満たすような$T_{\varepsilon}^{\dagger} \leq f(\varepsilon)$が存在する．**つまり，どのMDPに対しても劣線形リグレットになる時刻が多項式時間以内に訪れる．**
    * UCRLとTURCRLアルゴリズムはこれを満たします．
2. $\mathbb{E}\left[\Delta\left(\mathfrak{A}_T, M_0, x, T\right)\right] \leq C_2(\ln (T))^\beta$ for all $T \geq T_0^*$を満たす$T_0^*<+\infty$が存在する
    * SCALってやつがこれを満たす？これはAsymptoticな結果であることに注意しよう

つまり，２を満たすアルゴリズムがあったとしても，それは$\left.e^{D^{1 / \beta}}\right)$の線形リグレットに苛まれる可能性があります．
基本的には１を満たすアルゴリズムのほうが嬉しいです．直径が無限の場合は２のアルゴリズムはほぼ永遠に線形リグレットです．
